# Kimberlite TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data on a regular grid of a synthetic kimberlitic model.

In [377]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

### Auxiliary functions

In [378]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [379]:
model_dir = 'data/model.pickle'
grid_dir = '../grids/flightlines_grid_2100pts_100km2_-150m.pickle'
data_dir = 'data/flightlines_grid_2100pts_100k/data.pickle'

In [380]:
with open(model_dir) as w:
        model = pickle.load(w)

### Observation points and observed data

In [381]:
with open(grid_dir) as w:
        grid = pickle.load(w)

In [382]:
with open(data_dir) as w:
        data = pickle.load(w)

In [383]:
# observed data and observation points
dobs = data['tfa_obs']
xp = grid['x']
yp = grid['y']
zp = grid['z']
N = grid['N']

### Parameters of the initial model

In [384]:
M = 20 # number of vertices per prism
L = 5 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = model['inc']
decs = model['dec']

# depth to the top, thickness and radius
z0 = model['z0']
dz = 350.
r = 2000.

# total magnetization
intensity = model['intensity']
props = {'magnetization': utils.ang2vec(
        intensity, incs, decs)}
rin = np.zeros(M) + r
m0 = np.hstack((rin, np.zeros(2)))
m0 = np.resize(m0, P - 1) 
m0 = np.hstack((m0, dz)) # inicial parameters vector
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

# main field
inc, dec = data['main_field']

# predict data
d0 = polyprism.tf(xp, yp, zp, model0, inc, dec)

### Limits

In [385]:
# limits for parameters in meters
rmin = 10.
rmax = 3000.
x0min = -1000.
x0max = 1000.
y0min = -1000.
y0max = 1000.
dzmin = 10.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [386]:
# variation for derivatives
deltax = 0.01*np.max(x0max)
deltay = 0.01*np.max(y0max)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)

### Outcropping parameters

In [387]:
# outcropping body parameters
#m_out = np.zeros(M + 2)
m_out = model['param_vec'][:M+2]

### Folder to save the results

In [388]:
mypath = data_dir[:-11]+'tfa_inversion/inc%d_dec%d_r%d_int%d_z0%d_dz%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz)
if not os.path.isdir(mypath):
    os.makedirs(mypath)

In [389]:
# output of inversion
inversion = dict()

### Regularization parameters

In [390]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.1
dlamb = 10.      # step for Marquadt's parameter

a1 = 0.0001  # adjacent radial distances within each prism
a2 = 0.0003   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 0.0001     # vertically adjacent origins
a6 = 0.000001   # zero order Tikhonov on adjacent radial distances
a7 = 0.0001     # zero order Tikhonov on thickness of each prism

In [391]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [392]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

In [393]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [394]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['initial_estimate'] = model0
inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = alpha
inversion['tol'] = tol
inversion['main_field'] = [data['main_field'][0], data['main_field'][1]]

### Inversion

In [395]:
d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
    xp, yp, zp, m0, M, L, delta,
    itmax, itmax_marq, lamb,
    dlamb, tol, mmin, mmax,
    m_out, dobs, inc, dec,
    props, alpha, z0, dz
)

it:  0   it_marq:  0   lambda: 1e+01   misfit: 4.83042e+04
it:  1   it_marq:  0   lambda: 1e+00   misfit: 9.16138e+03
it:  2   it_marq:  0   lambda: 1e-01   misfit: 7.15729e+02
it:  3   it_marq:  0   lambda: 1e-02   misfit: 3.34201e+03
it:  3   it_marq:  1   lambda: 1e-01   misfit: 1.70353e+03
it:  3   it_marq:  2   lambda: 1e+00   misfit: 5.28080e+02
it:  4   it_marq:  0   lambda: 1e-01   misfit: 1.41954e+03
it:  4   it_marq:  1   lambda: 1e+00   misfit: 4.05469e+02
it:  5   it_marq:  0   lambda: 1e-01   misfit: 1.17234e+03
it:  5   it_marq:  1   lambda: 1e+00   misfit: 3.41245e+02
it:  6   it_marq:  0   lambda: 1e-01   misfit: 1.01697e+03
it:  6   it_marq:  1   lambda: 1e+00   misfit: 3.05400e+02
it:  7   it_marq:  0   lambda: 1e-01   misfit: 9.08753e+02
it:  7   it_marq:  1   lambda: 1e+00   misfit: 2.81416e+02
it:  8   it_marq:  0   lambda: 1e-01   misfit: 8.24128e+02
it:  8   it_marq:  1   lambda: 1e+00   misfit: 2.62916e+02
it:  9   it_marq:  0   lambda: 1e-01   misfit: 7.53289e+

# Results

In [396]:
inversion['data_fit'] = d_fit
inversion['estimate'] = m_est
inversion['prisms'] = model_est
inversion['estimated_models'] = model_list
inversion['objective'] = phi_list
inversion['residual'] = dobs - d_fit
inversion['residual_list'] = res_list

In [397]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)